# ML Project 2


## Imports and Such


In [26]:
import subprocess

# Installation on Google Colab
try:
    import os
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch', 'torchvision'])
    subprocess.run(['mkdir', '-p', 'datasets'])
    subprocess.run(['wget', '-nc', '--no-check-certificate',
                   'https://download.pytorch.org/tutorial/hymenoptera_data.zip', '-P', 'datasets'])
    subprocess.run(
        ['unzip', '-u', 'datasets/hymenoptera_data.zip', '-d' 'datasets'])
except ImportError:
    pass

In [27]:
import os
from urllib import request
from zipfile import ZipFile

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import datasets, models, transforms

from skorch import NeuralNetClassifier
from skorch.helper import predefined_split

torch.manual_seed(360)

## Loading dataset


In [28]:
data_dir = 'data/'
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])
val_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_ds = datasets.ImageFolder(
    os.path.join(data_dir, 'train'), train_transforms)
val_ds = datasets.ImageFolder(
    os.path.join(data_dir, 'valid'), val_transforms)

## Loading Pretrained Model

We use a pretrained `ResNet18` neural network model with its final layer replaced with a fully connected layer:


In [29]:
class PretrainedModel(nn.Module):
    def __init__(self, output_features):
        super().__init__()
        model = models.resnet18(pretrained=True)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, output_features)
        self.model = model

    def forward(self, x):
        return self.model(x)

### Callbacks


In [30]:
from skorch.callbacks import LRScheduler
from skorch.callbacks import Checkpoint
from skorch.callbacks import Freezer

lrscheduler = LRScheduler(
    policy='StepLR', step_size=7, gamma=0.1)

checkpoint = Checkpoint(
    f_params='best_model.pt', monitor='valid_acc_best')

freezer = Freezer(lambda x: not x.startswith('model.fc'))

### skorch.NeuralNetClassifier


In [31]:
net = NeuralNetClassifier(
    PretrainedModel,
    criterion=nn.CrossEntropyLoss,
    lr=0.001,
    batch_size=4,
    max_epochs=25,
    module__output_features=5,
    optimizer=optim.SGD,
    optimizer__momentum=0.9,
    iterator_train__shuffle=True,
    iterator_train__num_workers=2,
    iterator_valid__num_workers=2,
    train_split=predefined_split(val_ds),
    callbacks=[lrscheduler, checkpoint, freezer],
    classes=['Baseball', 'Basketball', 'Football', 'Hockey', 'Volleyball'],  # Fix for my error
    device='cpu' # Change to 'cuda' if you have a GPU
)

In [32]:
net.fit(train_ds, y=None)

  epoch    train_loss    valid_acc    valid_loss    cp      lr      dur
-------  ------------  -----------  ------------  ----  ------  -------
      1        1.3356       0.7644        0.6437     +  0.0010  46.8917
      2        0.9512       0.8712        0.3814     +  0.0010  46.2759
      3        0.7895       0.8685        0.3400        0.0010  46.7259
      4        0.7574       0.9315        0.2443     +  0.0010  46.6492
      5        0.7361       0.9178        0.2626        0.0010  51.5915
      6        0.8143       0.8767        0.3154        0.0010  47.0946
      7        0.7885       0.9342        0.2050     +  0.0010  45.7586
      8        0.6095       0.9370        0.2368     +  0.0001  47.2219
      9        0.6166       0.8849        0.2900        0.0001  46.2323
     10        0.5782       0.9370        0.2135        0.0001  45.9879
     11        0.6438       0.9397        0.2162     +  0.0001  44.9677
     12        0.6369       0.9507        0.1968     +  0.0001  

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=PretrainedModel(
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [38]:
# define ImageNet-based transformations (normalized with mean and standard deviation of ImageNet images)

from PIL import Image
model_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# define pretrained ResNet18 neural network model with final layer replaced with a fully connected
# layer (in using this, we will want to make the number of output features 2 since we trained
# a binary ant/bee classifer)

# create skorch.NeuralNetClassifier as in tutorial

# from skorch.callbacks import LRScheduler
# lrscheduler = LRScheduler(
#    policy='StepLR', step_size=7, gamma=0.1)
# from skorch.callbacks import Checkpoint
# checkpoint = Checkpoint(
#    f_params='best_model.pt', monitor='valid_acc_best')
# from skorch.callbacks import Freezer
# freezer = Freezer(lambda x: not x.startswith('model.fc'))


net = NeuralNetClassifier(
    PretrainedModel,
    criterion=nn.CrossEntropyLoss,
    module__output_features=5,
)

# LOAD MODEL FROM FILE
net.initialize()
net.load_params(f_params='best_model.pt')

# APPLY TO IMAGE
# image_path = 'antImage.jpg'
image_path = 'football_test.jpg'

# load image
img = Image.open(image_path)

# apply transformations
img_transformed = model_transforms(img).unsqueeze(0)

# apply model
classes = ['Baseball', 'Basketball', 'Football', 'Hockey', 'Volleyball']
output = net.predict(img_transformed)
# print(output)
print(classes[output[0]])

Football
